# Run PEST and Parallel PEST
### This should be super easy ~

In [6]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil
import sys
import pyemu
import flopy
import swatmf
assert "dependencies" in flopy.__file__
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatmf.__file__

In [7]:
from swatmf import swatmf_pst_par, swatmf_pst_utils

# 01. Specify a path to the model folder

In [8]:
prj_dir = "D:/test"

In [9]:
main_opt_path = os.path.join(prj_dir, 'main_opt')
os.chdir(main_opt_path)

# 2. Initial run for adjusting weights

In [8]:
pst_name = 'mb_zon.pst'
pyemu.os_utils.run(f"pestpp-glm {pst_name}",cwd=main_opt_path)

## 2.3 Change weights to make all of observation data visible 

In [9]:
# reweight
pst = pyemu.Pst(os.path.join(main_opt_path, pst_name))
pst.phi

3002.9504297625426

In [18]:
balanced_groups = {grp:1000 for grp in pst.nnz_obs_groups}
# print(pst.phi) #assumes "my.res" is colocated with "my.pst"
pst.adjust_weights(obsgrp_dict=balanced_groups)
pst.control_data.noptmax = 30

In [19]:
pst.write(os.path.join(main_opt_path,'mb_zon_rw.pst'), version=2)

noptmax:30, npar_adj:7, nnz_obs:305


# 3. Let's perform a history matching in parallel processing

In [12]:
num_workers = psutil.cpu_count(logical=False)
num_workers

16

In [13]:
m_d = os.path.join(prj_dir, "mb_zon_rw_glm")

In [14]:
os.chdir(prj_dir)

In [ ]:
pyemu.os_utils.start_workers(main_opt_path,"pestpp-glm",'mb_zon_rw.pst',
                            num_workers=num_workers,
                            worker_root='.',
                            master_dir=m_d)

In [ ]:
os.chdir(wd)

# 4. Let's see how we did:

In [ ]:
swatmf_viz.phi_progress_plot(pst_name_rw)

In [ ]:
pst = pyemu.Pst(os.path.join(wd, pst_name_rw))
pst.phi

In [ ]:
pst.res.loc[pst.nnz_obs_names,:]

In [ ]:
figs = pst.plot(kind="1to1")